# SRL Patient recognition experiments

In this notebook I carry out experiments to test whether two Semantic Role Labelling (SRL) systems can correctly identify patients in sentences with varying structures. This code was based on code provided by Pia Sommerauer.

In this code I load two models, namely the AllenNLP SRL model and the AllenNLP SRL BERT model. I create a variety of tets cases, for wich I evaluate the performance of the two models. All the test sentences are stored in a json file specified through the `test_sents_path` variable. The SRL predictions are stored in the json file specified through `srl_pred_path`, and similarly the SRL BERT predictions are stored at the path `bert_pred_path`.

### Recognizing locations as agents
In the news, location names like countries or capitols are often used as agents, to refer to the political power of that country, rather than to the country itself. This might be difficult for an SRL system if it uses Named Entity labels that recognize the names as locations. As the two models I test do not use this information directly, I expect the models can deal with this correctly.
* Location as location: 'The deal was made in Ukraine'
* Location as agent: 'Ukraine made a deal with Russia'


In [3]:
from allennlp_models.pretrained import load_predictor

In [4]:
import checklist
from checklist.editor import Editor
from checklist.perturb import Perturb
from checklist.test_types import MFT, INV, DIR
from checklist.expect import Expect

In [5]:
from checklist.pred_wrapper import PredictorWrapper

In [6]:
import json
import logging
logger = logging.getLogger()
logger.setLevel(logging.CRITICAL)

In [7]:
from utils_functions import *

### Load the model

In [8]:
# load the regular SRL model
srl_predictor = load_predictor('structured-prediction-srl')
# load the SRL BERT model
srlbert_predictor = load_predictor('structured-prediction-srl-bert')

2022-03-24 16:15:50,243 - INFO - allennlp.common.plugins - Plugin allennlp_models available
2022-03-24 16:15:50,336 - INFO - allennlp.common.plugins - Plugin allennlp_semparse available
2022-03-24 16:15:50,436 - INFO - allennlp.common.plugins - Plugin allennlp_server available
2022-03-24 16:15:50,470 - INFO - allennlp.common.params - id = pair-classification-esim
2022-03-24 16:15:50,470 - INFO - allennlp.common.params - registered_model_name = esim
2022-03-24 16:15:50,471 - INFO - allennlp.common.params - model_class = None
2022-03-24 16:15:50,472 - INFO - allennlp.common.params - registered_predictor_name = None
2022-03-24 16:15:50,473 - INFO - allennlp.common.params - display_name = Enhanced LSTM for Natural Language Inference
2022-03-24 16:15:50,474 - INFO - allennlp.common.params - task_id = textual_entailment
2022-03-24 16:15:50,475 - INFO - allennlp.common.params - model_usage.archive_file = esim-elmo-2020.11.11.tar.gz
2022-03-24 16:15:50,476 - INFO - allennlp.common.params - mod

2022-03-24 16:15:50,611 - INFO - allennlp.common.params - metrics.decision_thresholds = None
2022-03-24 16:15:50,612 - INFO - allennlp.common.params - metrics.variation_approaches = None
2022-03-24 16:15:50,613 - INFO - allennlp.common.params - evaluation_data.dataset = None
2022-03-24 16:15:50,615 - INFO - allennlp.common.params - evaluation_data.motivation = None
2022-03-24 16:15:50,617 - INFO - allennlp.common.params - evaluation_data.preprocessing = None
2022-03-24 16:15:50,619 - INFO - allennlp.common.params - training_data.dataset.name = DROP
2022-03-24 16:15:50,620 - INFO - allennlp.common.params - training_data.dataset.url = https://allennlp.org/drop
2022-03-24 16:15:50,621 - INFO - allennlp.common.params - training_data.motivation = None
2022-03-24 16:15:50,622 - INFO - allennlp.common.params - training_data.preprocessing = None
2022-03-24 16:15:50,624 - INFO - allennlp.common.params - quantitative_analyses.unitary_results = None
2022-03-24 16:15:50,625 - INFO - allennlp.commo

2022-03-24 16:15:50,779 - INFO - allennlp.common.params - model_details.version = 1
2022-03-24 16:15:50,780 - INFO - allennlp.common.params - model_details.model_type = RoBERTa large
2022-03-24 16:15:50,782 - INFO - allennlp.common.params - model_details.paper.citation = 
@article{Liu2019RoBERTaAR,
title={RoBERTa: A Robustly Optimized BERT Pretraining Approach},
author={Y. Liu and Myle Ott and Naman Goyal and Jingfei Du and Mandar Joshi and Danqi Chen and Omer Levy and M. Lewis and Luke Zettlemoyer and Veselin Stoyanov},
journal={ArXiv},
year={2019},
volume={abs/1907.11692}}

2022-03-24 16:15:50,783 - INFO - allennlp.common.params - model_details.paper.title = RoBERTa: A Robustly Optimized BERT Pretraining Approach (Liu et al)
2022-03-24 16:15:50,784 - INFO - allennlp.common.params - model_details.paper.url = https://api.semanticscholar.org/CorpusID:198953378
2022-03-24 16:15:50,785 - INFO - allennlp.common.params - model_details.license = None
2022-03-24 16:15:50,786 - INFO - allennlp

2022-03-24 16:15:50,940 - INFO - allennlp.common.params - model_class = None
2022-03-24 16:15:50,942 - INFO - allennlp.common.params - registered_predictor_name = None
2022-03-24 16:15:50,943 - INFO - allennlp.common.params - display_name = Coreference Resolution
2022-03-24 16:15:50,944 - INFO - allennlp.common.params - task_id = coref
2022-03-24 16:15:50,947 - INFO - allennlp.common.params - model_usage.archive_file = coref-spanbert-large-2021.03.10.tar.gz
2022-03-24 16:15:50,949 - INFO - allennlp.common.params - model_usage.training_config = coref/coref_spanbert_large.jsonnet
2022-03-24 16:15:50,950 - INFO - allennlp.common.params - model_usage.install_instructions = pip install allennlp==2.1.0 allennlp-models==2.1.0
2022-03-24 16:15:50,951 - INFO - allennlp.common.params - model_usage.overrides = None
2022-03-24 16:15:50,953 - INFO - allennlp.common.params - model_details.description = The basic outline of this model is to get an embedded representation of each span in the document.

2022-03-24 16:15:51,055 - INFO - allennlp.common.params - model_details.paper.url = https://api.semanticscholar.org/CorpusID:199453025
2022-03-24 16:15:51,056 - INFO - allennlp.common.params - model_details.license = None
2022-03-24 16:15:51,057 - INFO - allennlp.common.params - model_details.contact = allennlp-contact@allenai.org
2022-03-24 16:15:51,060 - INFO - allennlp.common.params - intended_use.primary_uses = This model is developed for the AllenNLP demo.
2022-03-24 16:15:51,061 - INFO - allennlp.common.params - intended_use.primary_users = None
2022-03-24 16:15:51,062 - INFO - allennlp.common.params - intended_use.out_of_scope_use_cases = None
2022-03-24 16:15:51,064 - INFO - allennlp.common.params - factors.relevant_factors = None
2022-03-24 16:15:51,065 - INFO - allennlp.common.params - factors.evaluation_factors = None
2022-03-24 16:15:51,067 - INFO - allennlp.common.params - metrics.model_performance_measures = Accuracy and F1-score
2022-03-24 16:15:51,068 - INFO - allennlp.

...

2022-03-24 16:15:59,056 - INFO - allennlp.common.params - model_details.developed_by = Lample et al
2022-03-24 16:15:59,057 - INFO - allennlp.common.params - model_details.contributed_by = None
2022-03-24 16:15:59,058 - INFO - allennlp.common.params - model_details.date = 2020-06-24
2022-03-24 16:15:59,059 - INFO - allennlp.common.params - model_details.version = 1
2022-03-24 16:15:59,059 - INFO - allennlp.common.params - model_details.model_type = BiLSTM
2022-03-24 16:15:59,061 - INFO - allennlp.common.params - model_details.paper.citation = 
@article{Lample2016NeuralAF,
title={Neural Architectures for Named Entity Recognition},
author={Guillaume Lample and Miguel Ballesteros and Sandeep Subramanian and K. Kawakami and Chris Dyer},
journal={ArXiv},
year={2016},
volume={abs/1603.01360}}

2022-03-24 16:15:59,062 - INFO - allennlp.common.params - model_details.paper.title = Neural Architectures for Named Entity Recognition
2022-03-24 16:15:59,063 - INFO - allennlp.common.params - model_d

2022-03-24 16:15:59,183 - INFO - allennlp.common.params - evaluation_data.preprocessing = None
2022-03-24 16:15:59,184 - INFO - allennlp.common.params - training_data.dataset.name = CNN/DailyMail
2022-03-24 16:15:59,185 - INFO - allennlp.common.params - training_data.dataset.notes = Please download the data from the url provided.
2022-03-24 16:15:59,186 - INFO - allennlp.common.params - training_data.dataset.url = https://github.com/abisee/cnn-dailymail
2022-03-24 16:15:59,186 - INFO - allennlp.common.params - training_data.motivation = None
2022-03-24 16:15:59,187 - INFO - allennlp.common.params - training_data.preprocessing = None
2022-03-24 16:15:59,189 - INFO - allennlp.common.params - quantitative_analyses.unitary_results = None
2022-03-24 16:15:59,190 - INFO - allennlp.common.params - quantitative_analyses.intersectional_results = None
2022-03-24 16:15:59,191 - INFO - allennlp.common.params - model_ethical_considerations.ethical_considerations = None
2022-03-24 16:15:59,192 - INF

2022-03-24 16:16:18,634 - INFO - allennlp.nn.initializers -    bert_model.encoder.layer.1.attention.output.LayerNorm.bias
2022-03-24 16:16:18,635 - INFO - allennlp.nn.initializers -    bert_model.encoder.layer.1.attention.output.LayerNorm.weight
2022-03-24 16:16:18,636 - INFO - allennlp.nn.initializers -    bert_model.encoder.layer.1.attention.output.dense.bias
2022-03-24 16:16:18,637 - INFO - allennlp.nn.initializers -    bert_model.encoder.layer.1.attention.output.dense.weight
2022-03-24 16:16:18,638 - INFO - allennlp.nn.initializers -    bert_model.encoder.layer.1.attention.self.key.bias
2022-03-24 16:16:18,639 - INFO - allennlp.nn.initializers -    bert_model.encoder.layer.1.attention.self.key.weight
2022-03-24 16:16:18,641 - INFO - allennlp.nn.initializers -    bert_model.encoder.layer.1.attention.self.query.bias
2022-03-24 16:16:18,643 - INFO - allennlp.nn.initializers -    bert_model.encoder.layer.1.attention.self.query.weight
2022-03-24 16:16:18,644 - INFO - allennlp.nn.initial

2022-03-24 16:16:18,704 - INFO - allennlp.nn.initializers -    bert_model.encoder.layer.3.attention.self.query.weight
2022-03-24 16:16:18,705 - INFO - allennlp.nn.initializers -    bert_model.encoder.layer.3.attention.self.value.bias
2022-03-24 16:16:18,706 - INFO - allennlp.nn.initializers -    bert_model.encoder.layer.3.attention.self.value.weight
2022-03-24 16:16:18,707 - INFO - allennlp.nn.initializers -    bert_model.encoder.layer.3.intermediate.dense.bias
2022-03-24 16:16:18,707 - INFO - allennlp.nn.initializers -    bert_model.encoder.layer.3.intermediate.dense.weight
2022-03-24 16:16:18,708 - INFO - allennlp.nn.initializers -    bert_model.encoder.layer.3.output.LayerNorm.bias
2022-03-24 16:16:18,711 - INFO - allennlp.nn.initializers -    bert_model.encoder.layer.3.output.LayerNorm.weight
2022-03-24 16:16:18,712 - INFO - allennlp.nn.initializers -    bert_model.encoder.layer.3.output.dense.bias
2022-03-24 16:16:18,712 - INFO - allennlp.nn.initializers -    bert_model.encoder.la

2022-03-24 16:16:18,768 - INFO - allennlp.nn.initializers -    bert_model.encoder.layer.7.output.dense.bias
2022-03-24 16:16:18,769 - INFO - allennlp.nn.initializers -    bert_model.encoder.layer.7.output.dense.weight
2022-03-24 16:16:18,769 - INFO - allennlp.nn.initializers -    bert_model.encoder.layer.8.attention.output.LayerNorm.bias
2022-03-24 16:16:18,770 - INFO - allennlp.nn.initializers -    bert_model.encoder.layer.8.attention.output.LayerNorm.weight
2022-03-24 16:16:18,771 - INFO - allennlp.nn.initializers -    bert_model.encoder.layer.8.attention.output.dense.bias
2022-03-24 16:16:18,772 - INFO - allennlp.nn.initializers -    bert_model.encoder.layer.8.attention.output.dense.weight
2022-03-24 16:16:18,773 - INFO - allennlp.nn.initializers -    bert_model.encoder.layer.8.attention.self.key.bias
2022-03-24 16:16:18,774 - INFO - allennlp.nn.initializers -    bert_model.encoder.layer.8.attention.self.key.weight
2022-03-24 16:16:18,777 - INFO - allennlp.nn.initializers -    bert_

In [9]:
#functions to create model predictions for a list containing sentences
### added by pia, edited by Goya ###

def predict_srl(data):
    pred = []
    for d in data:
        pred.append(srl_predictor.predict(d))
    return pred


def predict_srlbert(data):
    pred = []
    for d in data:
        pred.append(srlbert_predictor.predict(d))
    return pred

predict_srl = PredictorWrapper.wrap_predict(predict_srl)
predict_srlbert = PredictorWrapper.wrap_predict(predict_srlbert)

### Define output file paths

In [25]:
#create lists to store test sentences and model predictions in 
test_data = []
SRLBERT_predictions = []
SRL_predictions = []

In [10]:
#define paths to output files
test_sents_path = './JSON_test_and_predict_files/test_data_location.json'
bert_pred_path = './JSON_test_and_predict_files/BERT_predictions_location.json'
srl_pred_path = './JSON_test_and_predict_files/SRL_predictions_location.json'

#set name of current capability
capability = 'location_recognition'

### Load Checklist tests (Load functions defined in utils)
Load functions to test whether the arguments are correctly classified

In [2]:
expect_arg0_verb0 = Expect.single(found_location_arg0_verb0)
expect_arg0_verb1 = Expect.single(found_location_arg0_verb1)
expect_argloc_verb1 = Expect.single(found_location_argloc_verb1)

In [33]:
# initialize editor object
editor = Editor()
countries = [country for country in editor.lexicons.country if ('-' not in country and ',' not in country)]
cities = [city for city in editor.lexicons.city if ('-' not in city and ',' not in city)]

## Tests
Country as location

In [34]:
#create samples
testcase_name = 'country_location'
t = editor.template("The deal was made in {country}", country=countries, meta=True, nsamples=100)

#make and store predictions for the two models
test_data, SRL_predictions, SRLBERT_predictions = predict_and_store(t, capability, testcase_name, \
                                                                    expect_argloc_verb1, format_srl_verb1, \
                                                                    predict_srl, predict_srlbert, test_data, \
                                                                    SRL_predictions, SRLBERT_predictions)

SRL
Predicting 100 examples
Test cases:      100
Fails (rate):    1 (1.0%)

Example fails:
[ARG1: The deal] was [V: made] [ARGM-MNR: in Sierra Leone]
----
SRL BERT
Predicting 100 examples
Test cases:      100
Fails (rate):    0 (0.0%)


Country as agent

In [35]:
#create samples
testcase_name = 'country_agent'
t = editor.template("{country} made the deal", country= countries, meta=True, nsamples=100)

#make and store predictions for the two models
test_data, SRL_predictions, SRLBERT_predictions = predict_and_store(t, capability, testcase_name, \
                                                                    expect_arg0_verb0, format_srl_verb0, \
                                                                    predict_srl, predict_srlbert, test_data, \
                                                                    SRL_predictions, SRLBERT_predictions)

SRL
Predicting 100 examples
Test cases:      100
Fails (rate):    1 (1.0%)

Example fails:
[ARGM-MNR: Equatorial] [ARG0: Guinea] [V: made] [ARG1: the deal]
----
SRL BERT
Predicting 100 examples
Test cases:      100
Fails (rate):    0 (0.0%)


City as agent

In [36]:
#create samples
testcase_name = 'city_agent'
t = editor.template("{city} made the deal", city=cities, meta=True, nsamples=100)

#make and store predictions for the two models
test_data, SRL_predictions, SRLBERT_predictions = predict_and_store(t, capability, testcase_name, \
                                                                    expect_arg0_verb0, format_srl_verb0, \
                                                                    predict_srl, predict_srlbert, test_data, \
                                                                    SRL_predictions, SRLBERT_predictions)

SRL
Predicting 100 examples
Test cases:      100
Fails (rate):    0 (0.0%)
SRL BERT
Predicting 100 examples
Test cases:      100
Fails (rate):    0 (0.0%)


City as location, country as agent

In [37]:
#create samples
testcase_name = 'city_location_country_agent'
t = editor.template("In {city} the deal with the president was made by {country}", city=cities, country=countries, meta=True, nsamples=100)

#make and store predictions for the two models
test_data, SRL_predictions, SRLBERT_predictions = predict_and_store(t, capability, testcase_name, \
                                                                    expect_arg0_verb1, format_srl_verb1, \
                                                                    predict_srl, predict_srlbert, test_data, \
                                                                    SRL_predictions, SRLBERT_predictions)

SRL
Predicting 100 examples
Test cases:      100
Fails (rate):    1 (1.0%)

Example fails:
[ARGM-LOC: In Fort Wayne] [ARG1: the deal with the president] was [V: made] [ARGM-MNR: by Sierra Leone]
----
SRL BERT
Predicting 100 examples
Test cases:      100
Fails (rate):    0 (0.0%)


### Store all data to JSON

In [39]:
#store the test sentences
store_data(test_sents_path, test_data, new_file=True)
#store the model predictions
store_data(bert_pred_path, SRLBERT_predictions, new_file=True)
store_data(srl_pred_path, SRL_predictions, new_file=True)